# Agentic Memory System with Qdrant

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/agents/agentic_memory.ipynb)

Build an agent with persistent memory that learns from conversations. Uses Qdrant to store and retrieve memories across sessions.

**Requirements:** Set `OPENAI_API_KEY` environment variable.

In [ ]:
!pip install -q qdrant-client sentence-transformers openai

In [ ]:
import os
import time
import uuid
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import (
    PointStruct, VectorParams, Distance,
    Filter, FieldCondition, MatchValue,
    PayloadSchemaType,
)
from sentence_transformers import SentenceTransformer

In [ ]:
qdrant = QdrantClient(":memory:")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# Create memory collection with metadata indexes
qdrant.create_collection(
    collection_name="agent_memory",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

qdrant.create_payload_index(
    collection_name="agent_memory",
    field_name="memory_type",
    field_schema=PayloadSchemaType.KEYWORD,
)

qdrant.create_payload_index(
    collection_name="agent_memory",
    field_name="session_id",
    field_schema=PayloadSchemaType.KEYWORD,
)

print("Memory collection created")

In [ ]:
class AgentMemory:
    """Persistent memory system backed by Qdrant."""

    def __init__(self, client, embedder, collection="agent_memory"):
        self.client = client
        self.embedder = embedder
        self.collection = collection

    def store(self, text: str, memory_type: str, session_id: str, metadata: dict = None):
        """Store a memory with type tagging (fact, preference, conversation, etc)."""
        vector = self.embedder.encode(text).tolist()
        payload = {
            "text": text,
            "memory_type": memory_type,
            "session_id": session_id,
            "timestamp": time.time(),
            **(metadata or {}),
        }
        point_id = str(uuid.uuid4())
        self.client.upsert(
            collection_name=self.collection,
            points=[PointStruct(id=point_id, vector=vector, payload=payload)],
        )
        return point_id

    def recall(self, query: str, memory_type: str = None, limit: int = 5):
        """Retrieve relevant memories, optionally filtered by type."""
        vector = self.embedder.encode(query).tolist()
        query_filter = None
        if memory_type:
            query_filter = Filter(
                must=[FieldCondition(key="memory_type", match=MatchValue(value=memory_type))]
            )
        response = self.client.query_points(
            collection_name=self.collection,
            query=vector,
            query_filter=query_filter,
            limit=limit,
        )
        return [(p.payload["text"], p.score) for p in response.points]

memory = AgentMemory(qdrant, embedder)
print("Memory system initialized")

In [ ]:
# Simulate a conversation where the agent learns
session = "session_001"

# Store user preferences
memory.store("User prefers Python over JavaScript", "preference", session)
memory.store("User is building a RAG application for legal documents", "fact", session)
memory.store("User's Qdrant instance is on Qdrant Cloud with 2M vectors", "fact", session)
memory.store("User asked about quantization for memory optimization", "conversation", session)
memory.store("User prefers concise code examples over verbose explanations", "preference", session)

print("Stored 5 memories")

In [ ]:
# Agent uses memories to give personalized responses
def agent_respond(user_query: str):
    # Recall relevant memories
    all_memories = memory.recall(user_query, limit=3)
    preferences = memory.recall(user_query, memory_type="preference", limit=2)

    memory_context = "\n".join([f"- {text} (relevance: {score:.2f})" for text, score in all_memories])
    pref_context = "\n".join([f"- {text}" for text, _ in preferences])

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""You are a helpful Qdrant assistant with memory of past interactions.

Relevant memories:
{memory_context}

User preferences:
{pref_context}

Use these memories to personalize your response.""",
            },
            {"role": "user", "content": user_query},
        ],
    )

    answer = response.choices[0].message.content

    # Store this interaction as a new memory
    memory.store(f"User asked: {user_query}", "conversation", session)

    return answer, all_memories

answer, used_memories = agent_respond("How should I optimize my setup?")
print("Answer:", answer)
print("\nMemories used:")
for text, score in used_memories:
    print(f"  [{score:.2f}] {text}")